<a href="https://colab.research.google.com/github/darshita27-cmd/Face-Tracking-using-Arduino-Uno/blob/main/track.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import cv2
from cvzone.FaceDetectionModule import FaceDetector
import pyfirmata # it heps to controls the sesonrs, motors directly from python code
import numpy as np
import time


ws, hs = 1280, 720 # ws is the width of the video frame and hs is the height of the video frame in pixels

cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)

cap.set(3, ws) # 3 is the property identifier. 3 means width property is to be identified
cap.set(4, hs) # 4 is also a property identifier. 4 means height property is to me modified
time.sleep(2)

# Check if camera opened successfully
if not cap.isOpened():
    print("Camera could not be accessed. Exiting.")
    exit()
else:
    print("Camera initialized successfully.")

# Set up Arduino
port = "COM9"  # COM9 is series of ports in windows where ardino is connected. it could be for usb and could wary with different os
try:
    board = pyfirmata.Arduino(port)   # arduino borad is the hardware that will consist of micro cintroller. the pyfirmata library will help interact with the arduino board.
    it = pyfirmata.util.Iterator(board)
    it.start()
    print(f" Connected to Arduino on {port}")
except Exception as e:
    print(f" Failed to connect to Arduino on {port}: {e}")
    exit()

# Define servo pins
servo_pinX = board.get_pin('d:9:s')  # assesing the pins in arduino. d= digital pin. digital pins can be used as input or output both and can read or write the high or low binary values. 9 is the pin number. s=servo which means that this pin will e used for servo motors.
servo_pinY = board.get_pin('d:10:s')  # will control the servo motor with digital pin 10

# Face detector setup
detector = FaceDetector()
servoPos = [90, 90]   # sevoPos stores the list of values. it stores position or angles of the both servo_pinX and servo_pinY.

while True:
    success, img = cap.read() # cap return two values. success will have a boolean value to indicate if the frame was successfully captured. img will have the actual image in array from the video feed

    # Check for valid frame
    if not success or img is None:
        print(" Skipping frame: Unable to read from camera.")
        continue

    # Detect faces
    img, bboxs = detector.findFaces(img, draw=False) # draw =False means green boxes will not be drawn. img is the input image with faces. detector.FindFaces will return two values.  imag and bboxs. bboxes will have the bounding boxes corrdinates  to detect the faces. the bounding box returns [x,y, weight, height]

    if bboxs:
        fx, fy = bboxs[0]["center"] #[0], bboxs[0]["center"][1] # bboxs will get first element which will have bounding boxes value. center will have coordinates of the center of the face detected, [0] will get x postion (horizontal position in center) and [1] will get the y position (vertical in center)
        # pos=[fx,fy] # storing the center coordinates of the image in the pos variable
        servoX = np.interp(fx, [0, ws], [180, 0]) # interp performs the linear and take three parameters x ( input value), xp(coordinate of data point or the range of input values), fp(y coordinates of data points. the output values). fx represent x. fy represent y. [0,ws ] is for the x coordinates which are from 0 to the width. [180,0] is how much the servor motors range in which they can move
        servoY = np.interp(fy, [0, hs], [180, 0]) # if fx is 0 wich mean that facedetected is in leftmost position than server position will be 180
        servoX = np.clip(servoX, 0, 180)
        servoY = np.clip(servoY, 0, 180)

        servoPos[0] = servoX # getting the value of the angle needed to get in servoPos
        servoPos[1] = servoY

        cv2.circle(img, (fx, fy), 80, (0, 0, 255), 2) # drawing a circle on image. fx,fy are center coordinates. 80 is the radius.(0,0,255) gets color red. 2 is thickness of the circle
        cv2.putText(img, str((fx, fy)), (fx + 15, fy - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2) # adding text to image. str(pos) is the text to print.fx+15,fy-15 is position where text to be written. 2 is size of text,2 is thickness of text
        cv2.line(img, (0, fy), (ws, fy), (0, 0, 0), 2) # drawing horizontal line at y coordinate. (0,fy) is position to start starting the line. (ws,fy) is the ending point of the line. (0,0,0) color will be black. 2 is the thickness of the line
        cv2.line(img, (fx, hs), (fx, 0), (0, 0, 0), 2)
        cv2.circle(img, (fx, fy), 15, (0, 0, 255), cv2.FILLED) # drawing a filled circle
        cv2.putText(img, "TARGET LOCKED", (850, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    else:
        # No target visuals
        cv2.putText(img, "NO TARGET", (880, 50), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
        cv2.circle(img, (640, 360), 80, (0, 0, 255), 2)
        cv2.circle(img, (640, 360), 15, (0, 0, 255), cv2.FILLED)
        cv2.line(img, (0, 360), (ws, 360), (0, 0, 0), 2)
        cv2.line(img, (640, hs), (640, 0), (0, 0, 0), 2)


    cv2.putText(img, f'Servo X: {int(servoPos[0])} deg', (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2) # servo x:  {int(servoPos[0])}, deg gets the angle of x axis the servo motor holds. (50,50) is position where text will be written 50 pixels from left and 50pixels from right. 2 is the font size.
    cv2.putText(img, f'Servo Y: {int(servoPos[1])} deg', (50, 100), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)

    # Send to Arduino
    servo_pinX.write(servoPos[0]) # servoPos has the postion or needed angle in form of an array. write is used to send a command to servo motor to move in the specifioed direction
    servo_pinY.write(servoPos[1])

    time.sleep(0.01)  # Allow time for servos to respond

    # Show frame
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting on user command.")
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()